In [27]:
import cv2
import mediapipe as mp
import numpy as np
import os
import pandas as pd
import matplotlib.pyplot as plt
mp_drawing = mp.solutions.drawing_utils


In [6]:
data_train = pd.read_csv(os.path.join("..", "data", "revalidated", "data_train_f_001.csv"))
data_train.head()

,path,label,conf,revalidated_manually
0,.\data\train\subject-1\fall\1_backward_falls\f...,0,0.000000,1
1,.\data\train\subject-1\fall\1_backward_falls\f...,0,0.000000,1
2,.\data\train\subject-1\fall\1_backward_falls\f...,1,0.444923,1
3,.\data\train\subject-1\fall\1_backward_falls\f...,1,0.616470,1
4,.\data\train\subject-1\fall\1_backward_falls\f...,0,0.000000,1


In [47]:
def get_keypoints(image_path, visualize=False):
    mp_pose = mp.solutions.pose
    key_points = []
    image = cv2.imread(os.path.join("..", image_path))
    if image is None:
        raise FileNotFoundError(f"Image not found: {image_path}")
    
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    with mp_pose.Pose(min_detection_confidence=0.75, min_tracking_confidence=0.5) as pose:
        results = pose.process(image_rgb)
        
        if results.pose_landmarks:
            for idx, landmark in enumerate(results.pose_landmarks.landmark):
                # x, y: Koordinat 2D normalisasi.
                # z: Kedalaman relatif (persepsi 3D).
                # visibility: Keyakinan deteksi landmark.
                key_points.append({
                    'id': idx,
                    'x': landmark.x,
                    'y': landmark.y,
                    'z': landmark.z,
                    'visibility': landmark.visibility
                })
            
        if visualize:
            mp_drawing.draw_landmarks(
                image, 
                results.pose_landmarks, 
                mp_pose.POSE_CONNECTIONS,
                mp_drawing.DrawingSpec(color=(245, 117, 66), thickness=2, circle_radius=2),
                mp_drawing.DrawingSpec(color=(245, 66, 230), thickness=2, circle_radius=2)
            )
            cv2.imshow("Pose Detection", image)
            cv2.waitKey(0)
            cv2.destroyAllWindows()

    return key_points

def landmarks_to_vector(landmarks, with_visibility=False):
    vector = []
    for landmark in landmarks:
        vector.extend([landmark['x'], landmark['y'], landmark['z']])
        if with_visibility:
            vector.append(landmark['visibility'])
    return np.array(vector)

def plot_2d_landmarks(landmarks):
    x_coords = [lm['x'] for lm in landmarks]
    y_coords = [lm['y'] for lm in landmarks]

    plt.figure(figsize=(5, 5))
    plt.scatter(x_coords, y_coords, c='blue') 
    plt.plot(x_coords, y_coords, c='red', alpha=0.5)
    plt.gca().invert_yaxis()
    plt.title("2D Landmarks")
    plt.xlabel("X")
    plt.ylabel("Y")
    plt.show()

def denormalize_landmarks(landmarks, image_width, image_height):
    denormalized_landmarks = []
    for lm in landmarks:
        denormalized_landmarks.append({
            'id': lm['id'],
            'x': lm['x'] * image_width,
            'y': lm['y'] * image_height,
            'z': lm['z'],
            'visibility': lm['visibility']
        })
    return denormalized_landmarks

In [55]:
keypoints = []

for image_path in data_train.iloc[:, 0].values:
    keypoints.append(landmarks_to_vector(get_keypoints(image_path=image_path)))

In [ ]:
np.savez('arrays.npz', *keypoints)

In [69]:
keypoints[0]

array([ 0.52851665,  0.27551112, -0.11877798,  0.52542889,  0.2541835 ,
       -0.11212919,  0.52439618,  0.2526373 , -0.11216315,  0.52344882,
        0.25081885, -0.11219803,  0.5256356 ,  0.25525188, -0.13796447,
        0.52477622,  0.25459063, -0.13797744,  0.52402717,  0.25363967,
       -0.13803226,  0.5121972 ,  0.25159216, -0.0705405 ,  0.51375401,
        0.25465125, -0.18778557,  0.52285612,  0.2863062 , -0.09844121,
        0.52238733,  0.28730208, -0.13246642,  0.48786968,  0.32106602,
        0.0077727 ,  0.49484032,  0.33743665, -0.21864669,  0.48980492,
        0.43036258,  0.05195243,  0.5105443 ,  0.46103671, -0.23643973,
        0.50353795,  0.52402222,  0.03485499,  0.53305387,  0.56089896,
       -0.21687417,  0.50820369,  0.54383689,  0.03457958,  0.53612047,
        0.58608347, -0.24354801,  0.51116508,  0.54631215,  0.01578341,
        0.53983212,  0.58390272, -0.23680964,  0.50869977,  0.54181886,
        0.02554121,  0.53830838,  0.57380819, -0.21396618,  0.48